# Enhanced Backtest (Kalman Filter + Risk Controls)
Apply a Kalman Filter to estimate time-varying equilibrium and generate model-based z-score signals.


In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt, math
from pathlib import Path
df = pd.read_csv('../data/iv_spread_sample.csv', parse_dates=['date']).set_index('date')
y = df['iv_spread']


In [ ]:

def annualize_return(daily_returns, trading_days=252):
    daily_returns = daily_returns.fillna(0.0)
    cum = (1 + daily_returns).prod()
    years = len(daily_returns) / trading_days
    return cum**(1/years) - 1

def sharpe_ratio(daily_returns, trading_days=252):
    daily_returns = daily_returns.fillna(0.0)
    mu = daily_returns.mean()
    sigma = daily_returns.std(ddof=1)
    return 0.0 if sigma==0 else (mu * trading_days) / (sigma * np.sqrt(trading_days))

def max_drawdown(equity):
    eq = equity.fillna(method="ffill")
    peak = eq.cummax()
    dd = eq/peak - 1.0
    return float(dd.min())


In [ ]:

def kalman_filter(y, Q=0.02, R=0.12, huber_c=3.0):
    n = len(y)
    x = np.zeros(n); P = np.zeros(n); S_arr = np.zeros(n)
    x[0] = y.iloc[0]; P[0] = 1.0
    for i in range(1, n):
        x_pred = x[i-1]; P_pred = P[i-1] + Q
        if np.isnan(y.iloc[i]):
            x[i] = x_pred; P[i] = P_pred; S_arr[i] = P_pred + R; continue
        nu = y.iloc[i] - x_pred
        S = P_pred + R
        c = huber_c * math.sqrt(S)
        nu = min(max(nu, -c), c)
        K = P_pred / S
        x[i] = x_pred + K * nu
        P[i] = (1 - K) * P_pred
        S_arr[i] = S
    idx = y.index
    return pd.Series(x, index=idx), pd.Series(S_arr, index=idx)

def enhanced_backtest(y, entry_k=2.0, exit_k=0.5, stop_k=1.0, max_hold=10,
                      Q=0.02, R=0.12, huber_c=3.0, pnl_scale=0.0032):
    x, S = kalman_filter(y, Q=Q, R=R, huber_c=huber_c)
    z = (y - x) / np.sqrt(S)

    position = 0; hold = 0; entry_z = None
    daily_pnl = pd.Series(0.0, index=y.index)
    equity = pd.Series(1.0, index=y.index)
    prev_dev = (y.iloc[0] - x.iloc[0])

    for i in range(1, len(y)):
        dev = (y.iloc[i] - x.iloc[i]); dev_prev = prev_dev
        if position == 0:
            if z.iloc[i] > entry_k:
                position = +1; hold = 0; entry_z = z.iloc[i]
            elif z.iloc[i] < -entry_k:
                position = -1; hold = 0; entry_z = z.iloc[i]
        else:
            hold += 1
            if (position==+1 and z.iloc[i] > entry_z + stop_k) or (position==-1 and z.iloc[i] < entry_z - stop_k):
                position = 0; hold = 0; entry_z = None
            elif abs(z.iloc[i]) <= exit_k or hold >= max_hold:
                position = 0; hold = 0; entry_z = None

        if position != 0:
            daily_pnl.iloc[i] += position * (abs(dev_prev) - abs(dev)) * pnl_scale

        equity.iloc[i] = equity.iloc[i-1] * (1 + daily_pnl.iloc[i])
        prev_dev = dev

    return {"equity": equity, "daily_ret": daily_pnl, "x": x, "S": S, "z": z}


In [ ]:

res = enhanced_backtest(y)
print(f"Annualized Return: {annualize_return(res['daily_ret']):.2%}")
print(f"Sharpe Ratio     : {sharpe_ratio(res['daily_ret']):.2f}")
print(f"Max Drawdown     : {max_drawdown(res['equity']):.2%}")


In [ ]:

import matplotlib.pyplot as plt, numpy as np
# Equity
plt.figure(figsize=(9,4))
plt.plot(res["equity"].index, res["equity"].values)
plt.title("Enhanced Strategy Equity Curve (Kalman + Risk)")
plt.xlabel("Date"); plt.ylabel("Equity"); plt.tight_layout()
plt.show()

# Bands
x, S = res["x"], res["S"]
band = np.sqrt(S.values)
plt.figure(figsize=(9,4))
plt.plot(y.index, y.values, label='IV Spread')
plt.plot(x.index, x.values, label='Kalman Mean')
plt.plot(y.index, (x.values + 2*band), linestyle='--', label='+2√S')
plt.plot(y.index, (x.values - 2*band), linestyle='--', label='-2√S')
plt.title('IV Spread vs Kalman Equilibrium (±2√S)')
plt.xlabel('Date'); plt.ylabel('Spread'); plt.legend(); plt.tight_layout()
plt.show()

# Save for README
Path('../results').mkdir(exist_ok=True, parents=True)
plt.figure(figsize=(9,4))
plt.plot(res["equity"].index, res["equity"].values)
plt.title("Enhanced Strategy Equity Curve (Kalman + Risk)")
plt.xlabel("Date"); plt.ylabel("Equity"); plt.tight_layout()
plt.savefig('../results/enhanced_equity.png', dpi=150)
plt.close()

plt.figure(figsize=(9,4))
plt.plot(y.index, y.values, label='IV Spread')
plt.plot(x.index, x.values, label='Kalman Mean')
plt.plot(y.index, (x.values + 2*band), linestyle='--', label='+2√S')
plt.plot(y.index, (x.values - 2*band), linestyle='--', label='-2√S')
plt.title('IV Spread vs Kalman Equilibrium (±2√S)')
plt.xlabel('Date'); plt.ylabel('Spread'); plt.legend(); plt.tight_layout()
plt.savefig('../results/iv_spread_kalman_bands.png', dpi=150)
plt.close()
